In [1]:
import os


In [2]:
%pwd


'e:\\works\\chest_cancer-mlops\\Chest-Cancer-Detection-using-MLOps\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd


'e:\\works\\chest_cancer-mlops\\Chest-Cancer-Detection-using-MLOps'

In [5]:
import dagshub
dagshub.init(repo_owner='JeevanBiju', repo_name='Chest-Cancer-Detection-using-MLOps', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as JeevanBiju

Initialized MLflow to track repo "JeevanBiju/Chest-Cancer-Detection-using-MLOps"

Repository JeevanBiju/Chest-Cancer-Detection-using-MLOps initialized!

In [6]:
import tensorflow as tf


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from ChestCancerClassifier.constants import *
from ChestCancerClassifier.utils.common import read_yaml,create_directories,save_json

In [10]:

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/fall and nonfall",
            mlflow_uri="https://dagshub.com/JeevanBiju/Chest-Cancer-Detection-using-MLOps.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-10-02 19:17:39,058: INFO: common: yaml file: E:\works\chest_cancer-mlops\Chest-Cancer-Detection-using-MLOps\src\ChestCancerClassifier\config\config.yaml loaded successfully]
[2024-10-02 19:17:39,063: INFO: common: yaml file: E:\works\chest_cancer-mlops\Chest-Cancer-Detection-using-MLOps\src\ChestCancerClassifier\params.yaml loaded successfully]
[2024-10-02 19:17:39,065: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 11s 2s/step - loss: 26.6200 - accuracy: 0.4314
[2024-10-02 19:17:51,276: INFO: common: json file saved at: scores.json]
[2024-10-02 19:18:10,456: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /JeevanBiju/Chest-Cancer-Detection-using-MLOps.mlflow/api/2.0/mlflow/runs/cr

2024/10/02 19:19:03 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-10-02 19:19:07,634: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\jeeva\AppData\Local\Temp\tmpqnkrq4d0\model\data\model\assets
[2024-10-02 19:19:09,355: INFO: builder_impl: Assets written to: C:\Users\jeeva\AppData\Local\Temp\tmpqnkrq4d0\model\data\model\assets]


e:\Conda\envs\mlproj\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2024/10/02 19:20:52 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [1]:
import os
import random

# Set your folder path here
folder_path = 'E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male'

# Get list of all files in the folder
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Check if there are at least 3000 files
if len(all_files) < 3000:
    print(f"Folder contains only {len(all_files)} files. Cannot remove 3000 files.")
else:
    # Randomly select 3000 files to delete
    files_to_delete = random.sample(all_files, 3000)

    # Delete the files
    for file_path in files_to_delete:
        os.remove(file_path)
        print(f"Deleted: {file_path}")

    print("Successfully deleted 3000 random files.")


Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\brightness_2924_right_0_7543.jpg
Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\horizontal_flip_4570_right_0_7402.jpg
Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\shearing_2644_right_0_3831.jpg
Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\rotation_3273_left_0_807.jpg
Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\rotation_2207_right_0_8138.jpg
Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\zooming_1986_left_0_6522.jpg
Deleted: E:/works/Main Project/Fall Detection Using MLOps/artifacts/data_ingestion/Augmented images/Aug male\zooming_1230_left_0_639.jpg
Deleted: E:/works/